In [4]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['reviews']
result = mydb.reviews.create_index([('buyerId', pymongo.ASCENDING)],unique=False)
result = mydb.reviews.create_index([('productId', pymongo.ASCENDING)],unique=False)

In [5]:
import requests
base_url = 'https://m.aliexpress.com/ajaxapi/EvaluationSearchAjax.do?type=all&pageSize=80&country=US'
def importReviews(doc_complete, product_id):
    totalPage = 10000000000000000000
    n = 1
    while (n<=totalPage):
        #print(n)
        url = base_url + "&productId="+str(product_id)+"&index="+str(n)
        #print(url)
        resp = requests.get(url)
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET /EvaluationSearchAjax/ {}'.format(resp.status_code))
        resp_json = resp.json()
        doc_complete_n = [review["buyerTranslationFeedback"] 
                          for review in resp_json["evaViewList"] 
                          if "buyerTranslationFeedback" in review.keys() 
                          #and float(review["buyerEval"])<50
                         ]
        for review in resp_json["evaViewList"]:
            review["productId"] = product_id
            review["source"] = "aliexpress.com"
            try:
                mydb.reviews.insert_one(review)
            except: 
              pass
        doc_complete= doc_complete + doc_complete_n
        if (n==1):
            #print(resp_json)
            totalPage = int(resp_json["totalPage"])
            print(str(product_id)+" totalPage "+str(totalPage))
        print(".", end='')    
        n=n+1
    return doc_complete

In [6]:
#32800399551 doogee schießen 2 handy 2 gb - rating 2,7/5
#Xiaomi Redmi 4 Pro Prime
#  32752699189 Doogee x5 max 
# 32757712697 lenovo a5600 
# 32798494733 Lenovo ZUK Z2 
# 32791453530 ONEPLUS 3 T A3003 
# 32671947390  Blackview BV6000 
# 32719969124 Xiaomi Mi Max Mimax Pro 
# 32799364813 Oukitel U20 Plus Android 6.0
# 32611785137 Xiaomi Mi5 M5 Mi 5 Prime
product_ids = [32796300206,32763320059,32792320034,32766086429,32790980431,32721716490,32611785137,32769974242,32662806201]
doc_complete = []
for product_id in product_ids:
    doc_complete=importReviews(doc_complete,product_id)
print("count of reviews: "+ str(len(doc_complete)))



32796300206 totalPage 97
.................................................................................................32763320059 totalPage 107
...........................................................................................................32792320034 totalPage 106
..........................................................................................................32766086429 totalPage 63
...............................................................32790980431 totalPage 63
...............................................................32721716490 totalPage 60
............................................................32611785137 totalPage 52
....................................................32769974242 totalPage 50
..................................................32662806201 totalPage 43
...........................................count of reviews: 6644


In [8]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
#print(doc_clean)

In [9]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
#print(tagged)

In [11]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
#print(noun_phrase_tagged)
#doc_complete = nouns

In [13]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
#print(nouns)
doc_complete = nouns

In [14]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



[('phone', 4643), ('seller', 3124), ('day', 2265), ('everything', 1538), ('week', 1162), ('delivery', 1162), ('work', 1062), ('order', 1007), ('film', 808), ('time', 721), ('thank', 624), ('money', 621), ('gift', 524), ('quality', 459), ('recommend', 451), ('glass', 360), ('description', 360), ('problem', 358), ('case', 312), ('super', 309), ('track', 292), ('camera', 292), ('adapter', 290), ('screen', 289), ('purchase', 283), ('firmware', 283), ('region', 274), ('month', 274), ('product', 261), ('custom', 243), ('battery', 239), ('packaging', 225), ('price', 217), ('mail', 215), ('box', 205), ('\\\\_', 202), ('parcel', 189), ('thanks', 178), ('use', 169), ('smartphone', 168), ('russia', 164), ('advise', 161), ('shipping', 160), ('complaint', 148), ('store', 147), ('silicone', 139), ('post', 137), ('package', 137), ('fine', 135), ('dispute', 131), ('charge', 126), ('air', 122), ('cover', 121), ('way', 116), ('question', 111), ('extension', 110), ('photo', 109), ('star', 108), ('color',

def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [18]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

['smart phone recommend great phone', 'happy purchase band work perfect dial phone']
phone has absolute avg sentiment 0.40289711895910735
['happy purchase thank great phone seller', 'fine ecb 3 week put gift wrap order phone seller']
seller has absolute avg sentiment 0.4087893154190664
['everything tracked day packaging quality', 'excellent everything ok came 14 day']
day has absolute avg sentiment 0.2388339613970596
['fine seller forgot put adapter charger type c type everything', 'everything tracked day packaging quality']
everything has absolute avg sentiment 0.4027710273972607
['samara region two week', 'fine ecb 3 week put gift wrap order phone seller']
week has absolute avg sentiment 0.26532710526315806
['always super delivery 2 week quality', 'fine delivery 9 day']
delivery has absolute avg sentiment 0.3510705676855896
['store good good figment satisfied recommend work', 'model-the third work']
work has absolute avg sentiment 0.3732034383954153
['fine ecb 3 week put gift wrap or

['price class photo box already', 'дополнб review photo']
photo has absolute avg sentiment 0.3019247863247865
['three star jamb seller отправившего parcel mail russia', 'seller phone 5 star bravo']
star has absolute avg sentiment 0.24421376146788992
['already bag glass fact color black seller handsome', 'rose-gold beautiful color instead']
color has absolute avg sentiment 0.3353514563106794
['excellent everything ok came 14 day', 'seller-all excellent always phone']
excellent has absolute avg sentiment 0.6829231292517032
['write review', 'дополнб review photo']
review has absolute avg sentiment 0.28181727272727275
['phone safe sound', 'safe sound also fast gift wrap']
sound has absolute avg sentiment 0.461643571428571
['stavropol 12 day date order everything work fine firmware', 'fourth phone order aliekspress returned dispute 65 day date order seller']
date has absolute avg sentiment 0.25142680412371127
['loud conversation interlocutor hear clearly hand', 'well hand sealed securely']


In [19]:
print( [ (key,'{:01.2f}'.format(val)) for (key,val) in dict_noun_abs_sentiment.items() if val >0.1])

[('phone', '0.40'), ('seller', '0.41'), ('day', '0.24'), ('everything', '0.40'), ('week', '0.27'), ('delivery', '0.35'), ('work', '0.37'), ('order', '0.31'), ('film', '0.36'), ('time', '0.24'), ('thank', '0.61'), ('money', '0.36'), ('gift', '0.59'), ('quality', '0.43'), ('recommend', '0.58'), ('glass', '0.29'), ('description', '0.36'), ('problem', '0.37'), ('case', '0.34'), ('super', '0.71'), ('track', '0.27'), ('camera', '0.45'), ('adapter', '0.32'), ('screen', '0.32'), ('purchase', '0.39'), ('firmware', '0.29'), ('region', '0.22'), ('month', '0.25'), ('product', '0.43'), ('custom', '0.23'), ('battery', '0.34'), ('packaging', '0.43'), ('price', '0.43'), ('mail', '0.23'), ('box', '0.22'), ('\\\\_', '0.25'), ('parcel', '0.21'), ('thanks', '0.59'), ('use', '0.32'), ('smartphone', '0.42'), ('russia', '0.22'), ('advise', '0.36'), ('shipping', '0.36'), ('complaint', '0.33'), ('store', '0.39'), ('silicone', '0.35'), ('post', '0.17'), ('package', '0.24'), ('fine', '0.43'), ('dispute', '0.38')

# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean2)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean2]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=10)

print(ldamodel.print_topics(num_topics=50, num_words=5))

model = gensim.models.Word2Vec(doc_clean)

print(model)

model.similarity('camera', 'battery')